In [1]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi

pd.set_option('display.max_columns', 500)


In [2]:
from nba_api.stats.static import players
from nba_api.stats.static import teams

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
nba_teams = teams.get_teams()

nba_players = pd.DataFrame(nba_players)
nba_teams = pd.DataFrame(nba_teams)

active_plyaers = nba_players[nba_players['is_active']]

seasons = []
for i in list(range(1983,2023)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]
    
### 
testplayer='203076'
testgame = '0022201200'
testteam = '1610612755'

In [81]:
from nba_api.stats.endpoints import leaguegamefinder

result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]

listframe = []
for i in seasons[:10]:
    playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
    playerfinder['season']=i
    listframe.append(playerfinder)
    time.sleep(.5)
allplayers = pd.concat(listframe,axis=0)

listframe = []
for i in seasons[:10]:
    gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_type_nullable="Regular Season",season_nullable=i).get_data_frames()[0]
    listframe.append(gamefinder)
    time.sleep(1)
gamehistory = pd.concat(listframe,axis=0)
gamehistory['GAME_ID'] = pd.to_numeric(gamehistory['GAME_ID'])

In [3]:
bxv2 = nbaapi.boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=testgame).get_data_frames()


In [48]:
# wish list
# get a some score for last n games played 

#LeagueDashPlayerBioStats
#LeagueDashPtStats


In [102]:
### Accumulate game distance data for PLAYERS

# listframe = []
# for j in range(22013,22023):
#     ghdf = gamehistory[gamehistory['SEASON_ID']==str(j)]
#     for i in ghdf['GAME_ID'].drop_duplicates():
#         tempdf = nbaapi.boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=i).get_data_frames()[0]#[['GAME_ID','TEAM_ID','TEAM_ABBREVIATION','PLAYER_ID','PLAYER_NAME','MIN','SPD','DIST']]
#         listframe.append(tempdf)
#         time.sleep(2)
#     aggdf_player = pd.concat(listframe,axis=0)
#     print(j)
#     aggdf_player.to_csv('game_player_dist_{}.csv'.format(j),index=None)

### Accumulate game distance data for TEAMS

# listframe = []
# for i in gamehistory['GAME_ID'].drop_duplicates():
#     tempdf = nbaapi.boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=i).get_data_frames()[1][['GAME_ID','TEAM_ID','DIST']]
#     listframe.append(tempdf)
#     time.sleep(1)
# aggdf = pd.concat(listframe,axis=0)

In [104]:
# APPENDING ALL SEASON GAME PLAYER DIST FILES
# ldfs = []
# for j in range(22013,22023):
#     tdf = pd.read_csv('game_player_dist_{}.csv'.format(j))
#     ldfs.append(tdf)
# fdf = pd.concat(ldfs,axis=0)
# fdf.to_csv('game_player_dist.csv',index=None)

In [117]:
fdf.merge(gamehistory[['GAME_ID','GAME_DATE']].drop_duplicates(),on='GAME_ID',how='left').drop_duplicates(subset=['GAME_ID','TEAM_ID','PLAYER_ID']).to_csv('game_player_dist.csv',index=None)

In [98]:
df1 = aggdf.merge(gamehistory[['GAME_ID','TEAM_ID','MIN']],on=['GAME_ID','TEAM_ID'],how='left')

In [343]:
gamehistory['HOME'] = gamehistory.apply(lambda x: 'HOME' if '@' not in x['MATCHUP'] else 'AWAY',axis=1)
gamehistory['OPPONENT'] = gamehistory['MATCHUP'].str.split(' ',2).apply(lambda x: x[-1])
gamehistory['HOMEAWAY'] = gamehistory['MATCHUP'].apply(lambda x: 'HOME' if 'vs.' in x else 'AWAY')

gh = gamehistory[['GAME_ID','TEAM_ID','TEAM_ABBREVIATION','HOMEAWAY','OPPONENT','WL','PTS','MIN','MATCHUP']]

gh = gh.merge(highalt[['TEAM','HIGHALT']],left_on='TEAM_ABBREVIATION',right_on='TEAM',how='inner')
gh = gh.merge(highalt[['TEAM','HIGHALT']],left_on='OPPONENT',right_on='TEAM',how='inner',suffixes=['_home','_away'])

gh = gh.merge(highalt,left_on='TEAM_ABBREVIATION',right_on='TEAM',how='inner')
df1 = aggdf.merge(gh,on=['GAME_ID','TEAM_ID'],how='left').sort_values('GAME_ID')

In [367]:
df2 = df1[df1['MIN']<242]
df2 = df2[df2['DIST']>1]
df1.groupby(['TEAM_ABBREVIATION','HIGHALT_home','HIGHALT_away'])['DIST'].agg(np.mean).to_csv('team_meandisttravel_homeandaway.csv')
df2[['TEAM_ABBREVIATION','HIGHALT_home','HIGHALT_away','DIST']].to_csv('team_meandisttravel_homeandaway_nongrouped.csv')

In [310]:
# dfaway = df1[df1['HOMEAWAY']=='AWAY']
# dfaway['ALTDIF'] = dfaway['ALTITUDE_HOME'] - dfaway['ALTITUDE_AWAY']

# dfaway = dfaway[dfaway['DIST']!=0]
# dfaway['DISTperMIN'] = dfaway['DIST']/dfaway['MIN']
# dfaway.to_csv('forr1.csv',index=None)

In [51]:
#nbaapi.commonteamroster.CommonTeamRoster(team_id=testteam).get_data_frames()[0].sort_values('PLAYER_ID')

In [49]:
#nbaapi.playergamelogs.PlayerGameLogs().get_data_frames()[0]

In [50]:
#nbaapi.leaguegamelog.LeagueGameLog().get_data_frames()[0]